# TSP

#### An optimization algorithm by Qapitán

The TSP or Travelling Salesperson Problem is a classic optimization problem. The problem is, essentially, the following:

We are a Salesperson and we must visit several cities in order to sell all our products to as many people as possible. The cities are distant from each other, so we cannot sell anything in the middle of the travel. In addition, the travel makes us lose money since there are some expenses, manteinance and accomodation, to defray. Thus, we are interested in finding the route with the less possible travel time. 

This problem is known to be NP-Complete, that is, it is really hard to find the optimal solution to it. If we want to visit N cities, then we must find the optimal path among N! possible itineraries. Comparing one solution to another one is easy, but finding the best one is similar to looking for a needle in a haystack.

In this example, we need to determine a list of cities to visit. These cities should be the capitals of any country in the world. There are some additional calculations to extract the distances between all different pairs of cities. Then, a matrix with distances is introduced into the **Qapitán API**, who looks for the optimal solution in an easy way. The solution is showed afterwards. 

Take into account that the hardest piece of this problem is finding the optimal solution. This step is completely addressed by the **Qapitán**, while the users / cabin boys and girls do not have to worry about it. 


In [10]:
# Import some auxiliary packages. Make sure that all packages are installed

import pandas as pd
from geopy import distance 
import numpy as np
from Qapitan import Qapitan # Qapitan SDK


In [11]:
df = pd.read_csv("concap.csv")
df = df[['CapitalName', 'CapitalLatitude', 'CapitalLongitude']]
df.rename(columns={"CapitalName": "city", "CapitalLatitude": "lat", "CapitalLongitude": "lon"}, inplace=True)

In [12]:
# Introduce your cities here. Make sure they are capitals. If the cities are not in our database, an Error will
# raise in the next cell

cities = ['Berlin', 'Tokyo', 'Brasilia','Pretoria','Canberra']
starting_city = 'Tokyo'

In [13]:
df_city = df[df['city'].isin(cities)].reset_index()
assert(len(df_city) == len(cities)), "Some cities are not recognized!"

cities = df_city['city'] # Order is normalized
first_city = int(df_city[df_city['city'] == starting_city].index[0])

We compute now the distance matrix. Note that the distances are normalized to be not larger than 1, and the the diagonal terms are always -1. This is done to understand that the Traveler cannot go from one city to the same one. 

In [14]:
distances = np.zeros((len(cities),)*2)

for i in range(len(cities)):
    for j in range(i):
        distances[i, j] = distance.distance((df_city.loc[i, "lat"], df_city.loc[i, "lon"]), (df_city.loc[j, "lat"], df_city.loc[j, "lon"])).km
        distances[j, i]

distances = .5*(distances + distances.T)
distances/= np.max(distances)
distances -= np.eye(len(cities))

Now we are moving towards using the **Qapitán API** to solve the problem.

In [15]:
# Import required packages and stablish the parameters

import json
import requests

QAPITAN_PUBLIC_API= "https://hpg6m6hw7c.execute-api.eu-west-3.amazonaws.com/dev"
PAYLOAD_USER = {'username': 'adrian@qapitan.com', 'password': 'qapified'}

Now it is time to define the problem. In particular, the algorithm and provider must be specified. We are in the advent of quantum computing, and thus we will use simulations by now. We use an annealing scheme simulated using the DWave technology.

In [16]:
# Define the problem

PAYLOAD_TSP_1 = {
    "data": {
        "number_nodes": 3,
        "weight_matrix": [[-1,2,6], [2,-1,4],[6,4,-1]],
        "first_node": 0
    },
    "solvers":[
    {
        "name": "Solver_Qapitan_QUBO_Framework-Tsp-annealing_sim-dwave-local",
        "extra_arguments": {
            "num_reads": 1000,
            "chain_strength": 10
        }
    }
    ] 
}

In [17]:
qapitan_api = Qapitan(QAPITAN_PUBLIC_API, PAYLOAD_USER)
header = qapitan_api.login()

In [18]:
response_json = qapitan_api.execute(header=header, problem='tsp', payload=PAYLOAD_TSP_1)

In [19]:
response_json['job']

'7EF6O0PC40RX'

In [20]:
result = qapitan_api.get_result(header=header, job_name=response_json['job'])

In [21]:
print(result)

{'job': {'id': 8, 'created_at': '2021-06-29T09:12:18', 'end_at': None, 'details': None, 'name': 'SFJLF6XL7JCH', 'status': 'PENDING', 'user_id': 3}}


In [22]:
# Delete when integration is done:
import random
l1 = list(range(len(cities)))
random.shuffle(l1)

i = l1.index(first_city)

order = l1[i:] + l1[:i] + [first_city]

print(order)
#order is the expected return by the API

[3, 0, 4, 1, 2, 3]


We have now the optimal order to travel around the world! Let us paint it 

In [ ]:
# Let us include the order in the dataframe

df_city['order'] = [order.index(i) for i in range(len(cities))]



In [ ]:
df_city = df_city.sort_values(by=['order'])
route = ''
for city in df_city['city']:
    route += city + ', '
    
route += starting_city
print('The optimal route is \n', route)

The optimal route is 
 Tokyo, Berlin, Pretoria, Canberra, Brasilia, Tokyo


In [ ]:
import plotly.graph_objects as go
# Now we draw the route to follow. Notice that the red line stands for the travel back home
fig = go.Figure(data=go.Scattergeo(
    locationmode = 'USA-states',
        lon = df_city['lon'],
        lat = df_city['lat'],
        text = df_city['city'],
        mode = 'markers+text',
    name="cities",  showlegend=False, 
        ))

# draw the paths between the capitals
for i in range(len(cities)):
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon=[df_city.loc[order[i],"lon"],df_city.loc[order[i+1],"lon"]], 
        lat=[df_city.loc[order[i],"lat"],df_city.loc[order[i+1],"lat"]], 
        name="-".join([df_city.loc[order[i],"city"],df_city.loc[order[i+1],"city"]]),
        mode="lines", line_color="#000000", showlegend=False))
    
# the last path
fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon=[df_city.loc[order[-2],"lon"],df_city.loc[order[-1],"lon"]], 
        lat=[df_city.loc[order[-2],"lat"],df_city.loc[order[-1],"lat"]], 
        name="-".join([df_city.loc[order[-2],"city"],df_city.loc[order[-1],"city"]]),
        mode="lines", line_color="#ff0000", showlegend=False))

fig.update_layout(
        title = 'Shortest Route Between Cities',
    font_size=16
    )
fig.show()